In [28]:
import requests
from bs4 import BeautifulSoup
import re
from collections import Counter
import time
import sqlite3
import pandas as pd
from datetime import datetime

In [86]:
city = input("What city would you like to search from? ").title()


In [87]:
choice = input("(Choose option 1 or 2)\nWould like to see\n1. What your pay will be with your current skills\n2. What skills have the most pay in general\n")
data = []
data1 = []

if choice == '1':
    skills = input("Input your current skill(s)(if plural separate by a space): ").title()
    for i in range(1,6):
        url = f"http://api.adzuna.com/v1/api/jobs/gb/search/{i}?app_id=bbb231a1&app_key=ead6a75aff389175119f5fc5fd7d97cc&content-type=application/json"
        query_params = {
            "what_or": skills,
            "where": city
        }
        response = requests.get(url, params=query_params)
        print(response)
        json = response.json()
        data.append(json)

        time.sleep(2)
        

if choice == '2':
    skills = ["Python", "SQL", "Machine Learning", "AWS", "Java","JavaScript","C++","CSS","HTML","MATLAB","AI"]
    search_terms = "data, software, web development, engineer, tech"
    for i in range(1,6):
        url = f"http://api.adzuna.com/v1/api/jobs/gb/search/{i}?app_id=bbb231a1&app_key=ead6a75aff389175119f5fc5fd7d97cc&content-type=application/json"
        query_params = {
            "what_or": search_terms,
            "where": city
        }
        response = requests.get(url, params=query_params)
        print(response)
        json = response.json()
        data1.extend(json['results'])

        time.sleep(2)
    
    total_skill_counts = Counter()
    skill_pattern = r'\b(' + '|'.join(map(re.escape, skills)) + r')\b'
    for entry in data1:
        text = entry.get('description', "")
        if text:
            matches = re.findall(skill_pattern, text, flags=re.IGNORECASE)
            normalised_matches = [m for m in matches]
            total_skill_counts.update(normalised_matches)

try:
    dataframe = pd.DataFrame(total_skill_counts.items(), columns=['Skill', 'Count'])
    dataframe
except NameError:
    print("~~~~~~~~~Data Loaded Successfully~~~~~~~~~")




<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
~~~~~~~~~Data Loaded Successfully~~~~~~~~~


In [9]:
data

[{'count': 5535,
  '__CLASS__': 'Adzuna::API::Response::JobSearchResults',
  'mean': 80732.91,
  'results': [{'__CLASS__': 'Adzuna::API::Response::Job',
    'longitude': -0.02806,
    'latitude': 51.451818,
    'salary_min': 74709.04,
    'redirect_url': 'https://www.adzuna.co.uk/jobs/land/ad/5644291671?se=yCVeWN0S8RGJQOvVz3AcIQ&utm_medium=api&utm_source=bbb231a1&v=D55D180FEF2CFF0CD1DF26C043A5F8370E41B0E5',
    'salary_max': 74709.04,
    'description': 'Are you an experienced Python Developer with a passion for GIS and solving complex geospatial challenges? Join a team where youll build high-impact data pipelines, develop modern applications, and work with cutting-edge open-source tools to support large-scale spatial data operations. Were looking for someone who thrives in a collaborative environment, enjoys tackling technical challenges, and wants to contribute to meaningful spatial data solutions. What Youll Do As a core member of the engine…',
    'title': 'Python GIS Developer',
 

In [ ]:
try:
    min_list = []
    max_list = []
    jobs_list = []
    salary_list = []
    date_list = []
    company_list = []
    pattern = r"\d{4}-\d{2}-\d{2}"
    for a in range(5):
        sorted_jobs = sorted(
        data[a]['results'], 
        key=lambda x: datetime.fromisoformat(x['created']), 
        reverse=True
        )  
        for i in range(len(data[a]['results'])):
        
            salary_min = sorted_jobs[i]['salary_min']
            salary_max = sorted_jobs[i]['salary_max']
            max_list.append(salary_max)
            min_list.append(salary_min)

            date_text = sorted_jobs[i]['created']
            date = re.findall(pattern, date_text)
            date_list.append(date[0])

            count = data[0]['count']
            jobs = sorted_jobs[i]['title']
            company = sorted_jobs[i]['company']['display_name']
            jobs_list.append(jobs)
            salary_list.append(salary_min)
            company_list.append(company)

    estimate = ((sum(min_list)+sum(max_list))/(len(min_list)+len(max_list))) * 0.74 #GBP conversion
    print(f"Average Salary: £{estimate:.2f}")
    print(f"Amount of Job opprtunities requiring skill(s): {count}")
    print(company_list)

    database_data = []
    for job,salary,dates,company in zip(jobs_list,salary_list,date_list,company_list):
        job_data = (job, round(salary * 0.74, 2), dates,company)
        database_data.append(job_data)
except IndexError:
    print("~~~~~~~Halfway~~~~~~~")


Average Salary: £53782.71
Amount of Job opprtunities requiring skill(s): 5530
['Teksystems', 'mthree', 'Hyerhub', 'mthree', 'Hyerhub', 'Durlston Partners', 'Durlston Partners', 'Verve Talent Limited', 'Verve Talent Limited', 'Deloitte', 'Inspire Talent Ltd', 'Inspire Talent Ltd', 'Arrows', 'Arrows', 'THYME', 'THYME', 'Realm', 'Realm', 'Radley James', 'Radley James', 'Impax Recruitment', 'Impax Recruitment', 'Harrington Starr', 'Harrington Starr', 'Autonomai Recruitment', 'Autonomai Recruitment', 'CheMastery Group Ltd', 'CheMastery Group Ltd', 'Safe Intelligence', 'Safe Intelligence', 'Phi Partners', 'Phi Partners', 'Ascendion', 'Oakwell Hampton Group', 'Oakwell Hampton Group', 'Ascendion', 'Oliver Bernard', 'Oliver Bernard', 'Circuit Mind', 'Circuit Mind', 'Premier Group', 'Durlston Partners', 'Durlston Partners', 'Durlston Partners', 'Lendable', 'Solirius Reply', 'Oliver Bernard', 'Oliver Bernard', 'Deloitte', 'Enorth Staffing Limited']


In [95]:
try:
    conn = sqlite3.connect("Employment.db")
    conn.execute("PRAGMA foreign_keys = ON")

    cursor = conn.cursor()

    cursor.execute("""
                CREATE TABLE IF NOT EXISTS Jobs(
                Job_Id INTEGER PRIMARY KEY AUTOINCREMENT,
                Job_Title TEXT NOT NULL, 
                Salary INTEGER NOT NULL,
                Date_Posted DATE NOT NULL,
                Company TEXT NOT NULL
                )
                """)
    conn.commit()

    cursor.executemany("""
                    INSERT INTO Jobs (Job_Title, Salary, Date_Posted, Company)
                    VALUES (?, ?, ?, ?)
                    """, database_data)
    conn.commit()
except NameError:
    print("~~~~~~~Complete~~~~~~~")

In [94]:
cursor.execute("DROP TABLE IF EXISTS Jobs")
conn.commit()